In [ ]:
'''
IBN GRANITE MODELS
https://github.com/ibm-granite/granite-code-models


To use any of our models, pick an appropriate model_path from:
ibm-granite/granite-3b-code-base
ibm-granite/granite-3b-code-instruct
ibm-granite/granite-8b-code-base
ibm-granite/granite-8b-code-instruct
ibm-granite/granite-20b-code-base
ibm-granite/granite-20b-code-instruct
ibm-granite/granite-34b-code-base
ibm-granite/granite-34b-code-instruct
'''
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig

device = "cuda" # or "cpu"
model_path = "ibm-granite/granite-3b-code-instruct" # pick anyone from above list

tokenizer = AutoTokenizer.from_pretrained(model_path)
# Config added from repo to handle larger text inputs
config = AutoConfig.from_pretrained(model_path)
# config.max_seq_len = 4096
# config.max_answer_len= 1024
config.max_new_tokens = 250
config.max_length = 250

# drop device_map if running on CPU
model_code = AutoModelForCausalLM.from_pretrained(model_path
                                            #  , device_map=device
                                             , config = config)
model_code.eval()

In [ ]:
def get_information(model, question, device):
    input_tokens = tokenizer(question, return_tensors="pt")

    # transfer tokenized inputs to the device
    # for i in input_tokens:
    #     input_tokens[i] = input_tokens[i].to(device)

    # generate output tokens
    output = model.generate(**input_tokens)
    # decode output tokens into text
    output = tokenizer.batch_decode(output)
    return output

In [ ]:
test_text = "def generate():"
test_out = get_information(model_code, test_text, device)
# loop over the batch to print, in this example the batch size is 1
for i in test_out:
    print(i)

In [ ]:
base = '''what does this function do {}'''
example1 = base.format('''
even= []
uneven=[]
for i in INPUT:
  if i %2 ==0:
    even.append(i)
  else:
    uneven.append(i)
org = uneven + even
print(org)''')
# loop over the batch to print, in this example the batch size is 1
for i in get_information(model_code, example1, device):
    print(i)


In [ ]:
example2 = base.format('''
final = []
for item in INPUT:
  if item %2 ==0:
    final.insert(len(final)+1, item)
  else:
    final.insert(0, item)
print(final)''')
# loop over the batch to print, in this example the batch size is 1
for i in get_information(model_code, example2, device):
    print(i)

In [ ]:
example3 = base.format('''import pandas as pd
def create_df():
    data = {'state': ['Ohio','Ohio','Ohio','Nevada','Nevada'],
           'year': [2000,2001,2002,2001,2002],
           'pop': [1.5,1.7,3.6,2.4,2.9]}
    df = pd.DataFrame(data)
    return df

### We'll create three dataframes for an example
for i in range(3):
    exec(f'df_{i} = create_df()')''')
for i in get_information(model_code, example3, device):
    print(i)